In [1]:
import h5py
import numpy as np
import matplotlib.pyplot as plt
import numba as nb
import torch
import torch.nn.functional as F


c:\Users\Harsh\anaconda3\envs\bubble_sciml\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
twall_100 = h5py.File('Twall-100.hdf5', 'r')
[time_res, y_res, x_res] = twall_100['temperature'][:].shape

print(f'# Timesteps: {time_res}')
print(f'Domain resolution: {y_res} x {x_res}')

temp = twall_100['temperature'][:]
print(temp.shape)
velx = torch.from_numpy(twall_100['velx'][:]).cuda()
vely = torch.from_numpy(twall_100['vely'][:]).cuda()
pres = torch.from_numpy(twall_100['pressure'][:]).cuda()
print(pres.shape)
dfun = torch.from_numpy(twall_100['dfun'][:]).cuda()
x_dim = twall_100['x']
y_dim = twall_100['y']

param_dict = {}
for param in twall_100['real-runtime-params'][:]:
    param_dict[param[0].decode().strip()] = param[1]

# Timesteps: 201
Domain resolution: 48 x 48
(201, 48, 48)
torch.Size([201, 48, 48])


In [23]:
def trim_ends(field, dim):
        for d in dim:
            field = torch.narrow(field, d, 0, field.shape[d] - 1)
    
        return field

def compute_gradient(vel, resolution):
    grad = []
    
    for (i,res) in enumerate(resolution):
        dim = [-len(resolution)+((i+k)%len(resolution)) for k in range(1,len(resolution))]
        if i == 0:
            grad.append(trim_ends(compute_derivative(vel, res, -len(resolution)+i, -1), dim).unsqueeze(0))
        else:
            grad.append(trim_ends(compute_derivative(vel, res, -len(resolution)+i, 1), dim).unsqueeze(0))

    for i in grad:
        print(i.shape)
    grad = torch.cat(grad, dim = 0)

    return grad

    
def compute_derivative(field, resolution, dim, direction = 1):
    N = field.shape
    
    resh = [1] * len(N)
    resh[dim] = N[dim]

    reps = list(N)
    reps[dim] = 1
    
    k = (torch.fft.fftfreq(N[dim], resolution).reshape(resh).repeat(reps)*resolution).cuda()

    field_derivative = torch.fft.fft(field, dim = dim)

    if direction == 1:
        field_derivative = field_derivative * (torch.exp(2j * np.pi * k) - 1)
    else:
        field_derivative = field_derivative * (1 - torch.exp(-2j * np.pi * k))
    
    field_derivative = torch.narrow(field_derivative, dim, 0, N[dim]//2 + 1)
    field_derivative = torch.fft.irfft(field_derivative, dim = dim)

    if direction == 1:
        field_derivative = torch.narrow(field_derivative, dim, 0, N[dim]-1)/resolution
    else:
        # pass
        # print(dim)
        field_derivative = torch.narrow(field_derivative, dim, 1, N[dim]-1)/resolution

    return field_derivative



In [20]:
def compose_viscosity(dfun):        
    inv_Re = param_dict['ins_invreynolds']
    rho = param_dict['mph_rhogas']
    mu = param_dict['mph_mugas']
        
    viscosity = torch.ones(dfun.shape).cuda()
    viscosity *= inv_Re
    viscosity = torch.where(dfun > 0, viscosity*(mu/rho), viscosity)

    return viscosity

def compose_pressure(dfun):
    rho = param_dict['mph_rhogas']
        
    pressure = torch.ones(dfun.shape).cuda()
    pressure = torch.where(dfun > 0, pressure*(1/rho), pressure)

    return pressure
def trim_start(field, dim):
        for d in dim:
            field = torch.narrow(field, d, 1, field.shape[d]-1)
    
        return field

In [35]:
def velocity_equation2D(pf, vx, vy, df, resolution):
    combined_magnitude = torch.sqrt(vx**2 + vy**2)
    grad_mag = compute_gradient(combined_magnitude, resolution)
    combined_magnitude = trim_ends(combined_magnitude, [-3, -2, -1])

    du_dt = grad_mag[0]
    grad_vel = grad_mag[1:]

    u_grad_u = grad_vel * combined_magnitude


    grad_pressure = compute_gradient(pf, resolution)[1:]

    pres_const = compose_pressure(df)
    reps = [1]*(len(pres_const.shape)+1)
    reps[0] = 2
    pres_const.unsqueeze(0).repeat(reps)
    pres_const = trim_ends(pres_const, [-2, -1])
    pres_const = trim_start(pres_const, [-3])

    pressure_term =  pres_const.cuda() * grad_pressure

    visc_const = compose_viscosity(df)
    visc_const = trim_ends(visc_const, [-2, -1])
    visc_const = trim_start(visc_const, [-3])
    spatial_grad_scaled = visc_const.cuda()*grad_vel

    grad_2_x = compute_derivative(spatial_grad_scaled[-1], resolution[-1], -1)
    grad_2_y = compute_derivative(spatial_grad_scaled[-2], resolution[-2], -2)
    grad_2_x = trim_ends(grad_2_x, [-2])
    grad_2_y = trim_ends(grad_2_y, [-1])

    pressure_term1 = trim_ends(pressure_term[0], [-2, -1])
    pressure_term2 = trim_ends(pressure_term[1], [-2, -1])
    u_grad_u1 = trim_ends(u_grad_u[0], [-2, -1])
    u_grad_u2 = trim_ends(u_grad_u[1], [-2, -1])
    # du_dt = trim_ends(du_dt, [-2. -1])

    print(du_dt.shape)


    return (du_dt + u_grad_u1 + u_grad_u2 + pressure_term1 + pressure_term2 - grad_2_x - grad_2_y - 1j)




In [36]:
resolution = [1, 
              y_dim[0,1,0] - y_dim[0,0,0], 
              x_dim[0,0,1] - x_dim[0,0,0]]

p = pres[45:51, :, :]
vx = velx[45:51, :, :]
vy = vely[45:51, :, :]
df = dfun[45:51, :, :]

velocity_equation2D(p, vx, vy, df, resolution)


torch.Size([1, 5, 47, 47])
torch.Size([1, 5, 47, 47])
torch.Size([1, 5, 47, 47])
torch.Size([1, 5, 47, 47])
torch.Size([1, 5, 47, 47])
torch.Size([1, 5, 47, 47])


TypeError: tuple indices must be integers or slices, not float

In [ ]:
fig, ax = plt.subplots(1, 7, figsize=(12, 5))

data = {
    'Vel Mag': combined_magnitude[50].cpu().numpy(),
    'Pressure Grad. X': pressure_term[0, 50].cpu().numpy(),
    'Pressure Grad. Y': pressure_term[1, 50].cpu().numpy(),
    'Vel Grad. X': u_grad_u[0, 50].cpu().numpy(),
    'Vel Grad. Y': u_grad_u[1, 50].cpu().numpy(),
    'grad2x': grad_2_x[50].cpu().numpy(),
    'grad2y': grad_2_y[50].cpu().numpy(),
}

for idx, (key, im) in enumerate(data.items()):
    im = ax[idx].imshow(np.flipud(im))
    fig.colorbar(im, ax=ax[idx], shrink=0.5)
    ax[idx].set_title(key)
    ax[idx].set_xticks([])
    ax[idx].set_yticks([])